### 連線到 MySQL 資料庫

In [ ]:
from sqlalchemy import create_engine, text

# 寫成 class 就能建立多個實例，連線到不同資料庫
class MySQL_DB:
    def __init__(self, user: str, password: str, db_name: str, host: str="localhost", port: int=3306) -> None:
        self.host = host
        self.port = str(port)
        self.user = user
        self.password = password # 通常是放在環境變數
        self.db_name = db_name
        self.engine = create_engine(f'mysql+pymysql://{self.user}:{self.password}@{self.host}:{self.port}')
    
    def create_db(self) -> None:
        with self.engine.connect() as connection:
            connection.execute(text(f"CREATE DATABASE IF NOT EXISTS {self.db_name} DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"))
            self.engine = create_engine(f'mysql+pymysql://{self.user}:{self.password}@{self.host}:{self.port}/{self.db_name}')
            print(f"資料庫 '{self.db_name}' 建立成功！")    

# 連線到 example_db 資料庫
db = MySQL_DB(user='root', password='example', db_name='example_db')
db.create_db()

### 1. 讀取 CSV 然後存入 MySQL

In [ ]:
import pandas as pd
from os import path

# 讀取 interview.csv，使用 path.join 來支援不同作業系統
csv_file = path.join(".", "data", "interview.csv")

# 用 pandas 讀成 DataFrame
df = pd.read_csv(csv_file)

# 存入資料庫
df.to_sql('interview', con=db.engine, if_exists='replace', index=False)

# 印出資料庫裡的東西，確定有存進去
print(pd.read_sql('SELECT * FROM interview', con=db.engine))

### 2. 讀取 Mysql 匯入的資料並將使用者的名子末兩碼改成 OO

In [ ]:
df = pd.read_sql('SELECT * FROM interview', con=db.engine)
df["name"] = df["name"].str[0] + "OO"
df.to_sql('interview', con=db.engine, if_exists='replace', index=False)

# 印出資料庫裡的東西，確定有存進去
new_df = pd.read_sql('SELECT * FROM interview', con=db.engine)
print(new_df)